# Tutorial FFT 3D sequential

In this tutorial, we present how to use fluidfft to perform 3D fft in sequential.

In [1]:
import numpy as np
from fluidfft.fft3d import methods_seq
from fluidfft import import_fft_class

In [2]:
print(methods_seq)

['fft3d.with_fftw3d']


We import a class and instantiate it:

In [3]:
cls = import_fft_class('fft3d.with_fftw3d')

In [4]:
o = cls(4, 8, 12)

Let's have a look at the attribute of this objects.

In [5]:
print('\n'.join([name for name in dir(o) if not name.startswith('__')]))

build_invariant_arrayK_from_2d_indices12X
build_invariant_arrayX_from_2d_indices12X
comm
fft
fft_as_arg
gather_Xspace
get_dimX_K
get_k_adim_loc
get_local_size_X
get_seq_indices_first_K
get_shapeK_loc
get_shapeK_seq
get_shapeX_loc
get_shapeX_seq
get_short_name
ifft
ifft_as_arg
nb_proc
rank
run_benchs
run_tests
scatter_Xspace
sum_wavenumbers


Let's run a test and benchmark the fft and ifft functions directly from C++.

In [6]:
print(o.run_tests())

1


In [7]:
t1, t2 = o.run_benchs()
print('t_fft = {} s; t_ifft = {} s'.format(t1, t2))

t_fft = 1.15e-05 s; t_ifft = 5.899999999999999e-06 s


Let's understand how the data is stored:

In [8]:
print(o.get_dimX_K())

(0, 1, 2)


which means that for this class, in Fourier space, the data is not transposed...

Now we can get the non dimensional wavenumber in the first and second dimensions:

In [9]:
k0, k1, k2 = o.get_k_adim_loc()
print('k0:', k0)
print('k1:', k1)
print('k2:', k2)

k0: [ 0  1  2 -1]
k1: [ 0  1  2  3  4 -3 -2 -1]
k2: [0 1 2 3 4 5 6]


In [10]:
print(o.get_seq_indices_first_K())

(0, 0)


and check that the shapes of the array in one process are the same than in sequential (we are in sequential, there is only one process):

In [11]:
assert o.get_shapeX_loc() == o.get_shapeX_seq()
assert o.get_shapeK_loc() == o.get_shapeK_seq()

Now, let's compute fast Fourier transforms. We first initialize arrays:

In [12]:
a = np.ones(o.get_shapeX_loc())
a_fft = np.empty(o.get_shapeK_loc(), dtype=np.complex128)

If we do not have the array where to put the result we can do:

In [13]:
a_fft = o.fft(a)

If we already have the array where to put the result we can do:

In [14]:
o.fft_as_arg(a, a_fft)

And finally for the inverse Fourier transform:

In [15]:
a = o.ifft(a_fft)

In [16]:
o.ifft_as_arg(a_fft, a)